Data Class

In [ ]:
import random

class sentiment:
  NEGATIVE="NEGATIVE"
  NEUTRAL="NEUTRAL"
  POSITIVE="POSITIVE"
class Review:
  def __init__(self,text,score):
    self.text=text
    self.score=score
    self.sentiment=self.get_sentiment()

  def get_sentiment(self):
    if self.score<=2:
      return sentiment.NEGATIVE
    elif self.score==3:
      return sentiment.NEUTRAL
    else:#score of 4 or 5
      return sentiment.POSITIVE
class Reviewcontainer:
    def __init__(self,reviews):
        self.reviews=reviews

    def get_text(self): # This method needs to be defined inside the Reviewcontainer class.
        return [X.text for X in self.reviews]

    def get_sentiment(self):
        return [X.sentiment for X in self.reviews]


    def evenly_distribute(self):
        negative=list(filter(lambda x:x.sentiment==sentiment.NEGATIVE,self.reviews))
        positive=list(filter(lambda x:x.sentiment==sentiment.POSITIVE,self.reviews))
        positive_shrunk=positive[:len(negative)]
        self.reviews=negative+positive_shrunk
        random.shuffle(self.reviews)




Loading the data

In [ ]:
import json

file_name='/content/Books_small_10000.json'


reviews=[]
with open(file_name)as f:
  for line in f:
    review=json.loads(line)
    print(review['reviewText'])
    print(review['overall'])
    reviews.append(Review(review['reviewText'],review['overall']))

reviews[5].text


Streaming output truncated to the last 5000 lines.
Starts out predictable, turns unexpected and then runs away with your imagination! Loved it and look forward to the sequel!
5.0
This book started as a thrill ride and didn't disappoint.  The characters were revealed and built upon and they kept you moving throughout the story.  This is a definite add to your reading list!
4.0
Entertaining and amusing.  Sometimes a bit tedious with repetitiveness but overall a relaxing brainless re ad with a few historical facts thrown in.
4.0
This book was as good as the rest of them if not better. It will make you laugh and cry all in the page. Can't wait for the sixth and final installment although I wish the story didn't have to end.
5.0
I really enjoyed the premise of this book. When I stumbled across it, I was really excited to find a different kind of read.  It seems so many ya romance right now follow much the same formula only the names have changed (to protect the innocent ha!). And it was won

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

In [ ]:
len(reviews)

10000

**Data Preparation**

In [ ]:
from sklearn.model_selection import train_test_split
training,test= train_test_split(reviews,test_size=0.33,random_state=42)
training_cont=Reviewcontainer(training)
test_cont=Reviewcontainer(test)



In [ ]:
training_X=[x.text for x in training]
training_y=[x.sentiment for x in training]

test_X=[x.text for x in test]
test_y=[x.sentiment for x in test]

In [ ]:
training_cont.evenly_distribute()

train_X = training_cont.get_text()
train_y = training_cont.get_sentiment()

test_cont.evenly_distribute()
test_X = test_cont.get_text()
test_y = test_cont.get_sentiment()

print(train_y.count(sentiment.POSITIVE))
print(train_y.count(sentiment.NEGATIVE))

436
436


Bag of word

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

#This book is great
#This bookis so bad

vectorizer=TfidfVectorizer()
train_X_vectors= vectorizer.fit_transform(train_X)
test_X_vectors=vectorizer.transform(test_X)

print(train_X[0])
print(train_X_vectors[0].toarray())

train_X_vectors
train_y

OMG  I loved this series.  I fell in love with all of them.  I am sad to see it ended.  Wish there was more to this series.  Thnaks for a great story.
[[0. 0. 0. ... 0. 0. 0.]]


['POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',

**Model Selection**

Classification

In [ ]:
from sklearn import svm

clf_svm=svm.SVC(kernel='linear')

clf_svm.fit(train_X_vectors,train_y)
test_X[0]

test_X_vectors[0]

clf_svm.predict(test_X_vectors[0])

array(['NEGATIVE'], dtype='<U8')

Decision tree

In [ ]:
from sklearn.tree  import DecisionTreeClassifier

clf_dec=DecisionTreeClassifier()
clf_dec.fit(train_X_vectors,train_y)

clf_dec.predict(test_X_vectors[0])

array(['NEGATIVE'], dtype='<U8')

Navies Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

clf_gnb=GaussianNB()
clf_gnb.fit(train_X_vectors.toarray(),train_y)


clf_gnb.predict(test_X_vectors[0].toarray())

array(['POSITIVE'], dtype='<U8')

Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

clf_log=LogisticRegression()
clf_log.fit(train_X_vectors,train_y)

clf_log.predict(test_X_vectors[0])

array(['NEGATIVE'], dtype='<U8')

Model Evaluation

In [ ]:
#mean accuracy
print(clf_svm.score(test_X_vectors,test_y))
print(clf_dec.score(test_X_vectors,test_y))
print(clf_gnb.score(test_X_vectors.toarray(),test_y))
print(clf_log.score(test_X_vectors,test_y))

0.8076923076923077
0.6298076923076923
0.6610576923076923
0.8028846153846154


In [ ]:
#f1 score
from sklearn.metrics import f1_score
f1_score(test_y,clf_svm.predict(test_X_vectors),average=None,labels=[sentiment.POSITIVE,sentiment.NEUTRAL,sentiment.NEGATIVE])
#f1_score(test_y,clf_log.predict(test_X_vectors),average=None,labels=[sentiment.POSITIVE,sentiment.NEUTRAL,sentiment.NEGATIVE])

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


array([0.80582524, 0.        , 0.80952381])

In [ ]:
test_y.count(sentiment.NEGATIVE)

208

In [ ]:
test=['I thoroughly enjoyed this,5 star','bad book do not buy', 'horrible waste of time']
new_test=vectorizer.transform(test)
clf_svm.predict(new_test)

array(['POSITIVE', 'NEGATIVE', 'NEGATIVE'], dtype='<U8')

In [ ]:
test=['not great','bad book do not buy', 'horrible waste of time']
new_test=vectorizer.transform(test)
clf_svm.predict(new_test)

array(['NEGATIVE', 'NEGATIVE', 'NEGATIVE'], dtype='<U8')

**Tuning pur model (with grid search)**

In [ ]:
from sklearn. model_selection import GridSearchCV

parameters={'kernel':('linear','rbf'),'C':(1,4,8,16,32)}

svc=svm.SVC()
clf=GridSearchCV(svc,parameters,cv=5)
clf.fit(train_X_vectors,train_y)




GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('linear', 'rbf')})

In [ ]:
print(clf.score(test_X_vectors,test_y))


0.8197115384615384


Saving the model


In [ ]:
import pickle
with open('sentiment_classifier.pkl','wb')as f:
  pickle.dump(clf,f)

Load the model

In [ ]:
with open('sentiment_classifier.pkl','rb')as f:
  loaded_clf=pickle.load(f)

In [ ]:
print(test_X[0])

loaded_clf.predict(test_X_vectors[0])

Crystal Newbie like me. I was looking for help with learning the crystals by Identification and meaning. The problem is there is only one small picture of the crystal at the very end of the chapter on the crystal. It should have been a few big pictures of each crystal at the beginning of each description! While I read it it was nearly usless to me as far as a reference learning to ID book. Not impressed.


array(['NEGATIVE'], dtype='<U8')